# Plan na dziś

## Sprawy formalne

1. Regulamin laboratoriów
    * 3 nieusprawiedliwione nieobecności dozwolone,
    * 50 pkt: 25 pkt za aktywność, 25 za finalny projekt,
    * zaliczenie laboratoriów >= 25 pkt,
    * punkty za aktywność: 3-4 mini projekty w trakcie semestru po 5/10 pkt stricte związane z laboratorami. Każdy projekt to część podstawowa plus część bardziej zaawansowana.
    * finalny projekt: prezentacje na ostatnich laboratoriach, w połowie semestru ogłosimy szczegóły. Projekt: umiejętności z WDUM + ZMUM (praktyczne zadanie).

2. Środowisko programistyczne

* Google Colab i notebooki `ipynb`,
* alternatywy: Jupyter Notebooks na koncie PW, Kaggle notebooks,
* język i pakiety
    - Python3,
    - numpy, pandas, tensorflow (Keras),
    - pakiety mniej znane: konieczność użycia komendy `pip install`

## Ćwiczenia

1. Rozgrzewka programistyczna: generowanie liczb pseudolosowych, implementacja regresji.
2. Regresja liniowa jako prosta sieć neuronowa:
    * biblioteka `Keras`.
3. Ddtwarzalność wyników (research reproducibility).
4. Dopasowanie modelu i (nie)spodziewany problem.


# Rozgrzewka

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

np.set_printoptions(suppress=True)
print(tf.__version__)

2.11.0


In [2]:
rng = np.random.default_rng(2023)

Wygeneruj $100$ obserwacji pseudolosowych takich, że

\begin{equation}
Y = \beta_0 + \beta_1 \cdot X_1 + \beta_2 \cdot X_2 + \beta_3 \cdot X_3 + \epsilon,
\end{equation}

gdzie

* $\beta = (150, -4, 2.5, 0)$,
* $X_1 \sim \mathcal{N}(100, 10)$,
* $X_2 \sim \mathcal{N}(50, 5)$,
* $X_3 \sim \mathcal{N}(200, 20)$,
* $\epsilon \sim \mathcal{N}(0, 5)$.


[Zapoznaj się z tym przewodnikiem po generowaniu liczb pseudolosowych w `Numpy`](https://numpy.org/doc/stable/reference/random/generator.html)

Przypomnij sobie podstawy pracy z ramkami danych DataFrame z pakietu Pandas. Przydatne źródła: książka *Przetwarzanie i Analiza Danych w Języku Python*, lub bezpośrednio [dokumentacja Pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)

Szkielet rozwiązania

```python
data = pd.DataFrame(
    data=...,
    columns=...
)

data["Y"] = ...
```

# Zadanie 1 Rozwiązanie a)

Utworzenie ramki danych bezpośrednio z macierzy (np. `array`).

Tworzenie macierzy z losowymi liczbami z rozkładu normalnego $\mathcal{N}(100, 10)$ o wymiarze 5 x 1

In [3]:
przyklad = rng.normal(100, 10, size=(5, 1))

In [4]:
przyklad

array([[106.01721294],
       [111.5161897 ],
       [ 86.40537644],
       [102.22055335],
       [ 92.24132455]])

In [5]:
przyklad.shape

(5, 1)

Uwaga! Podanie argumentu `size=5` utworzy wektor, tzn. `array` o ksztalcie `array.shape` równym `(5,)`. **To nie jest to samo co macierz 5x1!**

In [6]:
rng.normal(100, 10, size=5).shape

(5,)

Potrzebujemy podać do `pd.DataFrame` argument `data` będący macierzą o wymiarze 100x4: potrzebujemy więc połączyć (~konkatenacja) osobno utworzone losowe macierze o wymiarach 100x1 w jedną macierz.

[Zapoznaj się z dokumentacją Numpy](https://numpy.org/doc/stable/user/absolute_beginners.html#adding-removing-and-sorting-elements).
Można użyc funkcji `np.concatenate`, można `np.hstack`.

In [7]:
data=np.hstack((
    rng.normal(100, 10, size=(100, 1)),
    rng.normal(50, 5, size=(100, 1)),
    rng.normal(200, 20, size=(100, 1)),
    rng.normal(0, 5, size=(100, 1))
))

In [8]:
data.shape

(100, 4)

Zauważ, że utworzenie losowych wektorów o długości 100, a nie macierzy o wymiarze 100x1, i ich konkatenacja nie utworzy macierzy o żądanym kształcie, a wektor o wymiarze (400,).

In [9]:
np.hstack((
    rng.normal(100, 10, size=(100)),
    rng.normal(50, 5, size=(100)),
    rng.normal(200, 20, size=(100)),
    rng.normal(0, 5, size=(100))
)).shape

(400,)

W końcu utwórzmy zbiór danych:

In [10]:
data = pd.DataFrame(data=data, columns=["X1", "X2", "X3", "e"])
# utwórz nową kolumnę w ramce danych
data["Y"] = 150 - 4*data.X1 + 2.5*data.X2 + data.e

In [11]:
data.head()

,X1,X2,X3,e,Y
0,100.501862,57.510833,174.176176,-0.698338,-108.928704
1,90.929564,45.634748,226.859124,-3.496010,-103.127395
2,101.321381,51.586683,188.653573,-1.236163,-127.554978
3,114.049025,60.035400,188.649950,0.815086,-155.292513
4,104.041020,40.453104,186.644548,7.283253,-157.748070


# Zadanie 1 Rozwiązanie b)

Możemy też utworzyć ramkę ze słownika, tu trzeba już podać wektory, a nie macierze (czyli argument `size=(100)`, a nie `size=(100,1)`.

In [12]:
data = pd.DataFrame({
    "X1": rng.normal(100, 10, size=(100)),
    "X2": rng.normal(50, 5, size=(100)),
    "X3": rng.normal(200, 20, size=(100)),
    "e": rng.normal(0, 5, size=(100))
})
data["Y"] = 150 - 4*data.X1 + 2.5*data.X2 + data.e

In [13]:
data.head()

,X1,X2,X3,e,Y
0,108.071809,55.706677,183.772875,-2.071775,-145.092318
1,98.903736,51.351310,153.402699,-2.288113,-119.524780
2,112.378543,44.268976,205.718847,-5.281330,-194.123063
3,95.316297,45.861517,204.102834,-8.316329,-124.927724
4,90.873422,48.598679,197.966181,1.601466,-90.395523


# Implementacja regresji liniowej jako problem ML

Cel: znalezienie optymalnego wektora parametrów
 $\beta = (\beta_0, \beta_1, \ldots, \beta_p)^T$.

W tym celu definiujemy funkcję ryzyka $L(\beta)$ i szukamy

$$
\begin{equation}
\hat{\beta} = \underset{\beta}{\text{arg min}} \quad
L(\beta)
\end{equation}
$$

Powszechnie stosowana funkcja ryzyka
([o przydatnych statystycznie właściwościach](https://en.wikipedia.org/wiki/Ordinary_least_squares#Properties)) to *Mean Squared Error* MSE

$$
\begin{align}
L(\beta) &= \text{MSE}(\beta; X, y)\\
&= \sum\limits_{i=1}^n (\hat{y}_i - y_i)^2 \\
&= \sum\limits_{i=1}^n ({\beta} \mathbf{x}_i - y_i)^2
\qquad \text{where }\mathbf{x}_i \in R^p
\\
&= {\lVert X {\beta} - Y \rVert}^2
= (X {\beta} - Y)^T (X {\beta} - Y)
\end{align}
$$


## Jak wyznaczyć $\hat{\beta}$?

Warunek konieczny pierwszego rzędu (*a first order necessary condition*)

\begin{equation}
\nabla_\beta L(\beta) = 0.
\end{equation}

Policzmy gradient

$$
\begin{equation}
\nabla_\beta L(\beta) = 
\nabla_\beta (X \beta - Y)^T (X \beta - Y) =
-2X^TY + 2X^TX\beta.
\end{equation}
$$

Przyrównująć gradient do $\mathbf{0}$, otrzymamy

\begin{equation}
\hat{\beta} = (X^T X)^{-1} X^T Y.
\end{equation}

Aby sprawdzić, że $\hat{\beta}$ to globalne minimum, musimy sprawdzić warunki drugiego rzędu (second-order conditions), tj. przeanalizować określoność Hesjanu (Hessian matrix). [(dla chętnych)](https://en.wikipedia.org/wiki/Gauss%E2%80%93Markov_theorem#Proof)

# Zadanie 2

Zaimplementuj klasę `Regression` z metodą `fit` i dopasuj model regresji liniowej do wysymulowanych danych `data`.

Użyj polecenia `print` żeby wyświetlić w konsoli wartości $\hat{\beta}$.

Szkielet rozwiązania

[dokumentacja dot. algebry liniowej w Numpy](https://numpy.org/doc/stable/reference/routines.linalg.html)

```python
class Regression:
    def __init__(self, ...)
        self.weights = None
        ...

    def fit(self, X, y):
        pass
```

Klas będziemy używać do reprezentowania oraz trenowania modelu, a następnie używania wytrenowanego modelu do predykcji.

Potrzebna wiedza: podręcznik *Przetwarzanie i analiza danych w języku Python*, rozdział 16.1. Dla bardziej dociekliwych [dokumentacja Pythona](https://docs.python.org/3/tutorial/classes.html#a-first-look-at-classes).

# Wskazówka

Użyj `X @ Y` do pomnożenia macierzy, użyj `np.linalg.inv` do odwrócenia macierzy.

# Zadanie 2 Rozwiązanie

In [14]:
class Regression:
    def __init__(self):
        self.weights = None

    def fit(self, X, y):
        self.weights = np.linalg.inv(X.T @ X) @ (X.T @ y)

In [19]:
regression1 = Regression()

regression1.fit(data.loc[:, ["X1", "X2", "X3"]], data.Y.values)

In [20]:
print(regression1.weights)

[-3.56491687  3.61565654  0.24972164]


Mała uwaga co do `pd.Series` i `np.array`:

In [21]:
type(data.Y)

pandas.core.series.Series

In [22]:
type(data.Y.values)

numpy.ndarray

obiekt `data.Y` to obiekt typu `pd.Series`. Dopiero dostanie się do jego atrybutu `values`, tj. `data.Y.values` pozwala nam na bezpośrednią pracę z numpy'owym `np.array`. Dla większości operacji nie ma to znaczenia, ale warto pamiętać o tym rozróżnieniu.

# Zadanie 3

W powyższym przykładzie nie zawarliśmy wyrazu wolnego *intercept* (lub *bias* w żargonie NN). Możemy dołączyć wyraz wolny poprzez rozszerzenie macierzy:

$X := [\mathbf{1} | X]$,

Zmodyfikuj metody `__init__` oraz `fit` klasy `Regression` tak, żeby użytkownik miał wybór: użyć intercept lub nie.

Szkielet rozwiązania

```python
class Regression:
    def __init__(self, intercept=False)
        self.weights = None
        self.intercept = intercept

    def fit(self, X, y):
        if self.intercept:
            pass
        pass
```

# Wskazówka



Użyj `np.ones` oraz `np.c_` aby uzupełnić macierz $X$ (a *design matrix*) o wektor jednostkowy $\mathbf{1} = (1, \ldots, 1)^T$.

# Zadanie 3 rozwiązanie

Zadanie 3 pozostało do zrobienia dla chętnych.

---

# Reprezentacja regresji liniowej jako prostej sieci neuronowej

Biblioteka, której będziemy używać do trenowania sieci neuronowych, to `Keras`, aktualnie zintegrowana z `Tensorflow`.

## Zadanie 4

[Zapoznaj się z krótkim wprowadzeniem do `Keras`](https://keras.io/about/).


# Zadanie 5

Zapoznaj się z poniższym schematem prostej sieci neuronowej, implementującej regresją.

![scheme](https://www.kamilkmita.com/zmum/lab-01-nn-scheme.png)

W powyższym modelu:
* $x$ to wektor kolumnowy `[[X1], [X2], [X3]]` (czyli macierz `np.array` o wymiarze 3x1),
* $W$ = `[[w1, w2, w3]]` to macierz o wymiarze 1x3,
* $b \in R$,
* $z=Wx + b$ to warstwa typu `tf.keras.layers.Dense`,
* funkcja aktywacji $\sigma = Id$, co zapewnia nam model liniowy.

Zauważmy, że w żargonie NN mówimy o "wagach" $w_j$, a nie współczynnikach $\beta_j$, zaś wyraz wolny $w_0$ to "bias" a nie "intercept".

Utwórz model `model1` korzystając z biblioteki `Keras`, korzystając ze szkieletu poniżej. Korzystaj z *Sequential API*.

```python
model1 = ...

model1.add(tf.keras.Input(shape=...))

model1.add(
    tf.keras.layers.Dense(
        ..., 
        activation=...
    )
)
```

Następnie użyj metody `summary`, żeby wypisać podsumowanie modelu, oraz wypisz wartości atrybutu `weights`: zainicjowaną losowo przez Keras macierz wag $W$.

# Zadanie 5: Rozwiązanie.

In [28]:
model1 = tf.keras.Sequential()

model1.add(tf.keras.Input(shape=(3,)))

model1.add(
    tf.keras.layers.Dense(
        1, 
        activation=tf.keras.activations.linear
    )
)

In [29]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 4         
                                                                 
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [30]:
model1.weights

[<tf.Variable 'dense/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[-0.9292261],
        [-1.0422198],
        [ 0.3999014]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

Zmienna `dense/kernel:0` zawiera wagi $w_1, w_2, w_3$, zmienna `dense/bias:0` zawiera wyraz wolny (*bias*).

# Reprodukowalność

Macierz wag $W$ jest generowana losowo, ale możemy proces ustalania wag wstępnych kontrolować (mniej lub bardziej) za pomocą modułu `tf.keras.initializers.<...>` [(link do dokumentacji Keras)](https://keras.io/api/layers/initializers/#creating-custom-initializers)

To jest ważne ze względu na:
- reprodukowalność wyników naukowych,
- analizę działania sieci neuronowych: będziemy porównywać nasze implementacje "od zera" z modelami stworzonymi przy użyciu Keras.



# Zadanie 6

Stwórz pseudolosowy wektor wag `random_tensor` w formie tensora za pomocą biblioteki Tensorflow.
Użyj ziarna losowania o wartości `2023`.

Wypisując tensor do konsoli, powinieneś otrzymać rezultat

```python
<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.3747068 ,  0.72808206, -0.7266839 ], dtype=float32)>
```

Następnie stwórz nowy `model2`, podobny do `model1`, z tą różnicą, że macierz wag $W$ ma zawierać wyraz wolny $w_0 = 0$ oraz resztę wag równą wartościom wektora `random_tensor`. Znajdź odpowiedni argument w klasie `tf.keras.layers.Dense`, za pomocą którego przekażesz wylosowany wektor wag.

Przekonaj się, że atrybut `model2.weights` jest równy wylosowanemu wektorowi `random_tensor`.

Schemat rozwiązania

```
random_tensor = tf.random.normal((3,), seed=2023)
random_tensor
```

```
model2 = tf.keras.Sequential()

model2.add(tf.keras.Input(shape=(3,)))

model2.add(
    tf.keras.layers.Dense(
        1, 
        activation=tf.keras.activations.linear,
        <odpowiedni_argument>=<odpowiednia_wartosc>
    )
)
```

# Zadanie 6 Rozwiązanie

In [31]:
random_tensor = tf.random.normal((3,), seed=2023)
random_tensor

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.3747068 ,  0.72808206, -0.7266839 ], dtype=float32)>

In [32]:
model2 = tf.keras.Sequential()

model2.add(tf.keras.Input(shape=(3,)))

model2.add(
    tf.keras.layers.Dense(
        1, 
        activation=tf.keras.activations.linear,
        kernel_initializer=tf.keras.initializers.Constant(random_tensor)
    )
)

In [33]:
model2.get_weights()

[array([[ 0.3747068 ],
        [ 0.72808206],
        [-0.7266839 ]], dtype=float32), array([0.], dtype=float32)]

Zauważmy, że wagi w utworzonym modelu (który jeszcze nie został wytrenowany w żaden sposób) odpowiadają wagom z wektora `random_tensor`. Ponadto, defaultowa wartość *bias* to 0 - więc nie musieliśmy niczego podawać podczas tworzenia modelu, żeby - zgodnie z instrukcją - zainicjować wartość tego parametru jako 0.

# Trenowanie sieci neuronowej

# Training neural network - przypomnienie

* Liczba obserwacji: $N = 100$,

* an epoch: jedna iteracja po zbiorze danych (każda z obserwacji raz bierze udział w uczeniu),

* a batch: a fraction $M < N$ of the data used for a single gradient calculation and weights update, e.g. $M = 20$.

Schemat trenowania sieci neuronowej 3 epochs/5 batches może być opisany za pomoca takiego pseudokodu:

```python
for epoch in range(3):
    for batch in range(5):
        data_for_training = data.loc[batch_index, :]
        learning_process() # updating weights in current `batch` iteration
```

Importujemy odpowiedni *callback* żeby otrzymać *verbose log* (więcej informacji wypisanych do konsoli podczas procesu uczenia).

In [34]:
from keras.callbacks import LambdaCallback
print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model2.layers[0].get_weights()))

Kompilujemy model z użyciem klasycznego ustawienia *optimizer*, czyli metody SGD (Stochastic Gradient Descent) oraz analizowanej wcześniej funkcji straty MSE.

In [35]:
model2.compile(optimizer='sgd', loss='mse')

Użyjmy ustawień:

* 5 epochs,
* batch size: 20,
* `shuffle=False`. Opcja `shuffle` zapewnia reprodukowalność, bo algorytm wybiera obserwacje do danego batchu według kolejności ze zbioru danych, a nie losowo.

Model trenujemy za pomocą metody `fit` wywołanej na modelu z odpowiednimi argumentami.

In [36]:
history = model2.fit(
    data.loc[:, ["X1", "X2", "X3"]], 
    data.loc[:, ["Y"]], 
    epochs=5, 
    verbose=1, 
    callbacks = [print_weights], 
    batch_size=20, 
    shuffle=False
)

Epoch 1/5
1/5 [=====>........................] - ETA: 2s - loss: 8523.9473[array([[-1.3680037e+14],
       [-7.0413330e+13],
       [-2.9133454e+14]], dtype=float32), array([-1.361757e+12], dtype=float32)]
5/5 [==============================] - 1s 4ms/step - loss: 932744709117911920145334272.0000
Epoch 2/5
1/5 [=====>........................] - ETA: 0s - loss: 5295980526525174653454672478601216.0000[array([[1.7095752e+29],
       [8.7994537e+28],
       [3.6407670e+29]], dtype=float32), array([1.7017687e+27], dtype=float32)]
5/5 [==============================] - 0s 6ms/step - loss: inf                                
Epoch 3/5
1/5 [=====>........................] - ETA: 0s - loss: inf[array([[nan],
       [nan],
       [nan]], dtype=float32), array([nan], dtype=float32)]
5/5 [==============================] - 0s 3ms/step - loss: nan
Epoch 4/5
1/5 [=====>........................] - ETA: 0s - loss: nan[array([[nan],
       [nan],
       [nan]], dtype=float32), array([nan], dtype=float32

# Co się stało?


Od tego zaczniemy kolejne zajęcia.

---